In [20]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
from scipy.stats import mode
import calendar
import random
import bisect
import xgboost
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer
%matplotlib inline
from scipy.stats import entropy,skew,kurtosis
import pickle
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
import random
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import log_loss
import warnings
warnings.filterwarnings("ignore")

In [21]:
train_data=pd.read_csv("train_users_2.csv")

In [22]:
test_data=pd.read_csv("test_users.csv")

In [23]:
sessions_data=pd.read_csv("sessions.csv")

In [29]:
def function_1(X):
    """
    This function is used for
    1.Preprocessing data
    2.Feature Engineering
      a)Feature Extraction
      b)Feature selection
    3.Modelling Data
    ------------------------------
    Function parameters:
    -------------------
    X: Raw Data(list of DataFrame)
    
    Returns:
    -------
    predicted_data:predictions(array)
    
    
    """
    testdata=X[0]
    sessionsdata=X[1]
    #loading all the pickle files of models
    dic=joblib.load('dictionary_file.pkl')
    
    #1.DATA PREPROCESSING#
    #testdata#
    #Replace outliers#
    testdata.loc[(testdata["age"]<19) | (testdata["age"]>71),'age']=33.0 #train median age=33.0
    
    #fill null values#
    for i in testdata.columns:
        if i!='age':
            testdata[i].fillna('NAN',inplace=True)
    
    #drop columns which are not useful-test data
    #dropped date first booking column
    testdata.drop(['date_first_booking'],axis=1,inplace=True)
    
    
    #sessionsdata#
    #fill null values#
    if isinstance(sessionsdata,pd.DataFrame)==True:
        for i in sessionsdata.columns:
            if i!='secs_elapsed':
                sessionsdata[i].fillna('NAN',inplace=True)#categorical feature
            else:
                sessionsdata['secs_elapsed'].fillna(0 ,inplace=True)#numerical feature
            
        #drop rows where userid is null value-sessionsdata
        sessionsdata.dropna(subset=['user_id'],inplace=True)
    
    #2.FEATURE ENGINEERING#
    #2.1 FEATURE EXTRACTION#
     #if there is sessions data or not
    if isinstance(sessionsdata,pd.DataFrame)==False:
        mod_session= -1*np.ones((testdata.shape[0],869)) #trained sessionsdata contains 870 features(exclude id of train data)
        mod_session=pd.DataFrame(mod_session)
        
    else:
        #sessions data
        #if an action is repeated <100,it is replaced with others caetgory
        action=joblib.load(dic['action'])
        v=[]
        for i in action.index:
            if(action[i]>100):
                v.append(i)
    
        #replaced as others category
        lst=[]
        for j in sessionsdata['action']:
            if j not in v:
                lst.append("Others")
            else:
                lst.append(j)
            
        sessionsdata['action']=lst
        
        #extracted action array/matrix of size (action vector,action frequency) from action column
        #where action frequency=no.of times actions appeared and its index is retrieved
        #suppose if we have action frequency variable
        #index action value counts:#
        #4      show        140
        #3      fill         10
    
        #2      search      20
        #1     requested     5
    
        #here considering it as matrix where rows are the users(action_vector) and columns are the indexes  of action frequencies
        #index1 index2 index3 index4 -colns
     
        #eg:15th user actions let say : search show search
    
        #now 15th row will be  [0 2 0 1]
        #search will be incremented twice and show only once
        
        action_frequency=joblib.load(dic['action_frequency'])
        
        actions_vector=sessionsdata.groupby('user_id')['action'].apply(lambda x:x.values)
        

        action_array=np.zeros((len(actions_vector),len(action_frequency)))
        
        #extracted action type array/matrix of size (action type vector,action type frequency) from action column#
        #fill null values with NAN as a seperate category
    
        sec_log=sessionsdata.groupby('user_id')['secs_elapsed'].apply(lambda x:x.values)
    

        action_array_sec=np.zeros((len(actions_vector),len(action_frequency)))
        for k,i in enumerate(actions_vector):
            for l,j in enumerate(i):
                action_array[k][action_frequency[j]]+=1
                action_array_sec[k][action_frequency[j]]+=sec_log[k][l]
        sec_log_array_ac=[]
        for i in range(action_array_sec.shape[0]):
            v=np.log(1+action_array_sec[i,:])
            sec_log_array_ac.append(v)
        sec_log_array_ac=np.array(sec_log_array_ac)  

        #extracted action detail array/matrix of size (action detail vector,action detail frequency) from action column#
    
        #fill null values with NAN as a seperate category
        
        action_detail_frequency=joblib.load(dic['action_detail_frequency'])
        
        actions_detail_vector=sessionsdata.groupby('user_id')['action_detail'].apply(lambda x:x.values)

        actions_detail_array=np.zeros((len(actions_detail_vector),len(action_detail_frequency)))

        action_detail_array_sec=np.zeros((len(actions_detail_vector),len(action_detail_frequency)))
    
        for k,i in enumerate(actions_detail_vector):
            for l,j in enumerate(i):
                actions_detail_array[k][action_detail_frequency[j]]+=1
                action_detail_array_sec[k][action_detail_frequency[j]]+=sec_log[k][l]
            
        sec_log_array_ac_detail=[]
        for i in range(action_detail_array_sec.shape[0]):
            v=np.log(1+action_detail_array_sec[i,:])
            sec_log_array_ac_detail.append(v)
        sec_log_array_ac_detail=np.array(sec_log_array_ac_detail)
    
            
        #extracted device array/matrix of size (device vector,device frequency) from action column#
        #fill null values with NAN as a seperate category
        
        device_frequency=joblib.load(dic['device_frequency'])
        
        device_vector=sessionsdata.groupby('user_id')['device_type'].apply(lambda x:x.values)
        device_array=np.zeros((len(device_vector),len(device_frequency)))
        device_array_sec=np.zeros((len(device_vector),len(device_frequency)))
    
        for k,i in enumerate(device_vector):
            for l,j in enumerate(i):
                device_array[k][device_frequency[j]]+=1
                device_array_sec[k][device_frequency[j]]+=sec_log[k][l]
        sec_log_array_device=[]
        for i in range(device_array_sec.shape[0]):
            v=np.log(1+device_array_sec[i,:])
            sec_log_array_device.append(v)
        sec_log_array_device=np.array(sec_log_array_device)  
            
        #extracted action type array/matrix of size (action type vector,action type frequency) from action column#
        #fill null values with NAN as a seperate category
        
        sec_log=sessionsdata.groupby('user_id')['secs_elapsed'].apply(lambda x:x.values)
        action_type_frequency = joblib.load(dic['action_type_frequency'])

        action_type_vector=sessionsdata.groupby('user_id')['action_type'].apply(lambda x:x.values)

        action_type_array=np.zeros((len(action_type_vector),len(action_type_frequency)))
        action_type_array_sec=np.zeros((len(action_type_vector),len(action_type_frequency)))
        for k,i in enumerate(action_type_vector):
            for l,j in enumerate(i):
                action_type_array[k][action_type_frequency[j]]+=1
                action_type_array_sec[k][action_type_frequency[j]]+=sec_log[k][l]
        sec_log_array_ac_type=[]
        for i in range(action_type_array_sec.shape[0]):
            v=np.log(1+action_type_array_sec[i,:])
            sec_log_array_ac_type.append(v)
        sec_log_array_ac_type=np.array(sec_log_array_ac_type)
        
        #unique actions
        # no of unique actions done by user

        unique_actions=sessionsdata.groupby('user_id')['action'].nunique()
        unique_length= sessionsdata.groupby('user_id')['action'].apply(lambda x:len(x.values))  
        #obtaining mean,std of value counts of actions done by user,no.of unqiue actions done by user of action column

        unique_action_vector=sessionsdata.groupby('user_id')['action'].apply(lambda x:(x.value_counts().values))
        unique_action_vector_mean=sessionsdata.groupby('user_id')['action'].apply(lambda x:np.mean(x.value_counts().values))
        unique_action_vector_std=sessionsdata.groupby('user_id')['action'].apply(lambda x:np.std(x.value_counts().values))
        unique_actions=sessionsdata.groupby('user_id')['action'].apply(lambda x: x.nunique())
    
        #entropy,skew,kurtosis
        unique_action_vector_entropy=sessionsdata.groupby('user_id')['action'].apply(lambda x:entropy(x.value_counts().values))
        unique_action_vector_skew=sessionsdata.groupby('user_id')['action'].apply(lambda x:skew(x.value_counts().values))
        unique_action_vector_kurtosis=sessionsdata.groupby('user_id')['action'].apply(lambda x:kurtosis(x.value_counts().values))

        #obtaining mean,std of value counts of actions done by user,no.of unqiue actions done by user of action detail column

        unique_detail_vector=sessionsdata.groupby('user_id')['action_detail'].apply(lambda x:(x.value_counts().values))
        unique_detail_vector_mean=sessionsdata.groupby('user_id')['action_detail'].apply(lambda x:np.mean(x.value_counts().values))
        unique_detail_vector_std=sessionsdata.groupby('user_id')['action_detail'].apply(lambda x:np.std(x.value_counts().values))
        unique_action_detail=sessionsdata.groupby('user_id')['action_detail'].apply(lambda x: x.nunique())
    
        #entropy,skew,kurtosis
        unique_detail_vector_entropy=sessionsdata.groupby('user_id')['action_detail'].apply(lambda x:entropy(x.value_counts().values))
        unique_detail_vector_skew=sessionsdata.groupby('user_id')['action_detail'].apply(lambda x:skew(x.value_counts().values))
        unique_detail_vector_kurtosis=sessionsdata.groupby('user_id')['action_detail'].apply(lambda x:kurtosis(x.value_counts().values))
    
        #obtaining mean,std of value counts of actions done by user,no.of unqiue actions done by user of device type column

        unique_device_vector=sessionsdata.groupby('user_id')['device_type'].apply(lambda x:(x.value_counts().values))
        unique_device_vector_mean=sessionsdata.groupby('user_id')['device_type'].apply(lambda x:np.mean(x.value_counts().values))
        unique_device_vector_std=sessionsdata.groupby('user_id')['device_type'].apply(lambda x:np.std(x.value_counts().values))
        unique_devices=sessionsdata.groupby('user_id')['device_type'].apply(lambda x: x.nunique())
    
        #entropy,skew,kurtosis of device type
        unique_device_vector_entropy=sessionsdata.groupby('user_id')['device_type'].apply(lambda x:entropy(x.value_counts().values))
        unique_device_vector_skew=sessionsdata.groupby('user_id')['device_type'].apply(lambda x:skew(x.value_counts().values))
        unique_device_vector_kurtosis=sessionsdata.groupby('user_id')['device_type'].apply(lambda x:kurtosis(x.value_counts().values))
    
        #obtaining mean,std of value counts of actions done by user,no.of unqiue actions done by user of action type column

        unique_ac_type_vector=sessionsdata.groupby('user_id')['action_type'].apply(lambda x:(x.value_counts().values))
        unique_ac_type_vector_mean=sessionsdata.groupby('user_id')['action_type'].apply(lambda x:np.mean(x.value_counts().values))
        unique_ac_type_vector_std=sessionsdata.groupby('user_id')['action_type'].apply(lambda x:np.std(x.value_counts().values))
        unique_ac_type=sessionsdata.groupby('user_id')['action_type'].apply(lambda x: x.nunique()) 
    
        #entropy,skew,kurtosis of action type
        unique_ac_type_vector_entropy=sessionsdata.groupby('user_id')['action_type'].apply(lambda x:entropy(x.value_counts().values))
        unique_ac_type_vector_skew=sessionsdata.groupby('user_id')['action_type'].apply(lambda x:skew(x.value_counts().values))
        unique_ac_type_vector_kurtosis=sessionsdata.groupby('user_id')['action_type'].apply(lambda x:kurtosis(x.value_counts().values))

        #extracted log values of sum of secs elapsed column
        secs_elapsed_sum=sessionsdata.groupby('user_id')['secs_elapsed'].sum()
        secs_elapsed_sum_log=np.log(1+secs_elapsed_sum.values)
    
         #extracted log values of min of secs elapsed column
        #secs_elapsed_min=sessions_data.groupby('user_id')['secs_elapsed'].min()
        #secs_elapsed_min_log=np.log(1+secs_elapsed_min.values).astype(int)
    
        #extracted log values of max of secs elapsed column
       # secs_elapsed_max=sessions_data.groupby('user_id')['secs_elapsed'].max()
       # secs_elapsed_max_log=np.log(1+secs_elapsed_max.values).astype(int)
    
        #extracted log values of mean of secs elapsed column
        secs_elapsed_mean=sessionsdata.groupby('user_id')['secs_elapsed'].mean()
        secs_elapsed_mean_log=np.log(1+secs_elapsed_mean.values)
    
         #extracted log values of median of secs elapsed column
        secs_elapsed_median=sessionsdata.groupby('user_id')['secs_elapsed'].median()
        secs_elapsed_median_log=np.log(1+secs_elapsed_median.values)

        #extracted log values of std of secs elapsed column
        secs_elapsed_std=sessionsdata.groupby('user_id')['secs_elapsed'].std()
        secs_elapsed_std_log=np.log(1+secs_elapsed_std.values)

    

        #extracted bin counts of secs elapsed column
        log_secs=sessionsdata.groupby('user_id')['secs_elapsed'].apply(lambda x:np.log(1+x.values).astype(int))

        bin_feat=[]
        for i in log_secs:
            bin_feat.append(np.bincount(i,minlength=15).tolist())
    
        binfeat_arr=np.array(bin_feat)

        cols_bin=[]
        for i in range(binfeat_arr.shape[1]):
            cols_bin.append(binfeat_arr[:,i])
    
        #concatenated all the above extracted features
        all_arrays=np.hstack((action_array,actions_detail_array,device_array,action_type_array,sec_log_array_ac,sec_log_array_ac_detail,sec_log_array_device,sec_log_array_ac_type))

        #creating a dataframe for all the unique actions,action name,secs elapsed done by the user
        unique_session=pd.DataFrame()
        
        
        unique_session['id']=unique_actions.index
        unique_session['secs_elapsed_sum']=secs_elapsed_sum_log
        unique_session['secs_elapsed_mean']=secs_elapsed_mean_log
        unique_session['secs_elapsed_mean_sum']=secs_elapsed_sum_log/(secs_elapsed_mean_log).astype("float")
        # unique_session['secs_elapsed_min']=secs_elapsed_min_log
        # unique_session['secs_elapsed_max']=secs_elapsed_max_log
        unique_session['secs_elapsed_median']=secs_elapsed_median_log
        unique_session['secs_elapsed_std']=secs_elapsed_std_log
    
        unique_session['unique_action_vector_mean']=unique_action_vector_mean.values
        unique_session['unique_action_vector_std']=unique_action_vector_std.values
        unique_session['unique_actions']=unique_actions.values
    
        unique_session['unique_action_vector_entropy']=unique_action_vector_entropy.values
        unique_session['unique_action_vector_skew']=unique_action_vector_skew.values
        unique_session['unique_action_vector_kurtosis']=unique_action_vector_kurtosis.values
    

        unique_session['unique_detail_vector_mean']=unique_detail_vector_mean.values
        unique_session['unique_detail_vector_std']=unique_detail_vector_std.values
        unique_session['unique_action_detail']=unique_action_detail.values
    
        unique_session['unique_detail_vector_entropy']=unique_detail_vector_entropy.values
        unique_session['unique_detail_vector_skew']=unique_detail_vector_skew.values
        unique_session['unique_detail_vector_kurtosis']=unique_detail_vector_kurtosis.values
  

        unique_session['unique_device_vector_mean']=unique_device_vector_mean.values
        unique_session['unique_device_vector_std']=unique_device_vector_std.values
        unique_session['unique_devices']=unique_devices.values
    
        unique_session['unique_device_vector_entropy']=unique_device_vector_entropy.values
        unique_session['unique_device_vector_skew']=unique_device_vector_skew.values
        unique_session['unique_device_vector_kurtosis']=unique_device_vector_kurtosis.values

        unique_session['unique_ac_type_vector_mean']=unique_ac_type_vector_mean.values
        unique_session['unique_ac_type_vector_std']=unique_ac_type_vector_std.values
        unique_session['unique_ac_type']=unique_ac_type.values
    
        unique_session['unique_ac_type_vector_entropy']=unique_ac_type_vector_entropy.values
        unique_session['unique_ac_type_vector_skew']=unique_ac_type_vector_skew.values
        unique_session['unique_ac_type_vector_kurtosis']=unique_ac_type_vector_kurtosis.values
        unique_session['len']=unique_length.values
        
        for i in range(len(cols_bin)):
            unique_session["bin_secs"+str(i)]=cols_bin[i]
    
        all_array=pd.DataFrame(all_arrays) 

        #final preprocessed sessions dataframe
        final_unique_sessions=pd.concat((unique_session,all_array),axis=1)
        
    #testdata
    #timestamp first active#
    testdata['timestamp_first_active']=testdata['timestamp_first_active'].apply(str)
    testdata['timestamp_first_active']=pd.to_datetime(testdata['timestamp_first_active'])
    testdata['day_name']=testdata['timestamp_first_active'].apply(lambda x:calendar.day_name[x.weekday()])
    testdata['months']=testdata['timestamp_first_active'].dt.month
    testdata['day']=testdata['timestamp_first_active'].dt.day
    testdata['year']=testdata['timestamp_first_active'].dt.year
    
    #date accont created#
    testdata['date_account_created']=testdata['date_account_created'].apply(str)
    testdata['date_account_created']=pd.to_datetime(testdata['date_account_created'])
    testdata['dc_day_name']=testdata['date_account_created'].apply(lambda x:calendar.day_name[x.weekday()])
    testdata['dc_months']=testdata['date_account_created'].dt.month
    testdata['dc_day']=testdata['date_account_created'].dt.day
    testdata['dc_year']=testdata['date_account_created'].dt.year
    
    
    #difference of date acc created-timestamp first active #
    c=testdata['date_account_created']
    d=testdata['timestamp_first_active']

    testdata["diff"]=np.log(1+abs((c.dt.date-d.dt.date).dt.total_seconds()))
    testdata['sign_diff']=np.sign((c.dt.date-d.dt.date).dt.total_seconds())
    
    #quarters-date account created,time stamp first active#
    testdata['q_date']=testdata['date_account_created'].dt.quarter
    testdata['q_time']=testdata['timestamp_first_active'].dt.quarter 
    
    #drop date acc created and timestamp columns
    testdata.drop(['date_account_created','timestamp_first_active'],axis=1,inplace=True)
    
    #age#
    testdata['age_bins']=np.NaN
    
    #dividing age into 5 groups
    testdata.loc[(testdata['age']>=19)&(testdata['age']<=25),'age_bins']='young'
    testdata.loc[(testdata['age']>25)&(testdata['age']<=30),'age_bins']='adults'
    testdata.loc[(testdata['age']>30)&(testdata['age']<=40),'age_bins']='thirtes'
    testdata.loc[(testdata['age']>40)&(testdata['age']<=50),'age_bins']='middle_age'
    testdata.loc[testdata['age']>50,'age_bins']='elderly'
    
    testdata["age_bins"].fillna("NAN",inplace=True)

    
    #drop age column
    testdata.drop(['age'],axis=1,inplace=True)
    
    #gender#
    testdata = testdata.replace('-unknown-', "NAN")
    
    #no.of null values#
    testdata['n_null'] = np.array([sum(i == 'NAN') for i in testdata.values])
    
    categorical_features= ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 
                        'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser','age_bins','dc_day_name','day_name']
    
    #one hotencoding of categorical features
    def one_hot(testdata,column):
        enc=joblib.load(dic[column])
        enc_test=enc.transform(testdata[column].values.reshape(-1,1)).toarray()
    
        for i in range(enc_test.shape[1]):
            testdata[column+str(i)]=enc_test[:,i]
    for i in categorical_features:
        one_hot(testdata,i)
        testdata.drop(i, axis=1,inplace=True)
    
    #if there is sessions data or not
    if isinstance(sessionsdata,pd.DataFrame)==False:
        
        total_testdata=pd.concat((testdata,mod_session),axis=1)
    else:
        
        total_testdata=pd.merge(testdata,final_unique_sessions,how='left',on='id')
    
    total_testdata.fillna(-1,inplace=True)
    
    #drop id column
    user_id=total_testdata['id']
    total_test_data=total_testdata.drop(['id'],axis=1)
    
    #scaling total testdata
    scaler=joblib.load(dic['scaler'])
    
    scaled_data=scaler.transform(total_test_data)
    
    #2.2 FEATURE SELECTION
    
    #load the feature selection model-xgb
    xgb_feature_selection=joblib.load(dic['xgb_feature_selection'])
    
    # getting feature importance for each
    imp_scores = xgb_feature_selection.get_fscore()

    # mapping important feature value with features
    Imp_Features = np.zeros(scaled_data.shape[1])
    for k,v in imp_scores.items():
        
        Imp_Features[int(k[1:])] = v

    # normalization of feature importance    
    Imp_Features = Imp_Features/float(np.max(Imp_Features))

    # finding thresold to select important feature
    score=Imp_Features
    thresold = np.sort(score)[::-1][int(len(score)*0.7)] # selecting top 70% fetaures
    Imp_Features = score > thresold
    
    # Re-initilaizing data to keep only important feature
    imp_test_data = scaled_data[:, Imp_Features]
    
    #3.MODELLING#
    
    #best model-xgb 8000 estimators
    xgb_model=joblib.load('xgb_model')
    
    # evalute model 
    
    predicted_data = xgb_model.predict_proba(imp_test_data)
    
    return predicted_data

In [30]:
def function_2(X,Y):
    """
    This function is used to compute ndcg metric score for given raw data and true labels
    
    Function parameters:
    ------------------- 
    X: Raw data (list of dataframe )
    Y:True labels(array)
    
    Returns:
    -------
    score: Metric-Ndcg score(float)
    
    """
    
    #call fnction1 to get predictions
    y_pred=function_1(X)
    
    #encoded data of labels
    enc=joblib.load('label_enc.pkl')
    y_true=enc.transform(Y)
    
    
    #https://www.kaggle.com/davidgasquez/ndcg-scorer
    def dcg_score(y_true, y_score, k=5):
    
        """
        Compute DCG@k Score for an input
    
        parameters
        ----------
        y_true <ndarray>  : correct relevance values  
        y_score <ndarray> : predicted scores
        k <int>           : rank
    
        returns
        -------
        dcg_score <float> : DCG@5 score
    
        """ 
    
        # Compute releveace values for predictions
        order = np.argsort(y_score)[::-1]
        y_true = np.take(y_true, order[:k])
    
        # compute DCG@k for a given point
        dcg_numerator   = 2 ** y_true - 1
        dcg_denominator = np.log2( np.arange( len(y_true) ) + 2 )
        dcg_score = np.sum( dcg_numerator / dcg_denominator )
    
        return dcg_score
    def ndcg_score(ground_truth, predictions, k=5):
    
        """
        Compute NDCG@k Score for given inputs
    
        parameters
        ----------
        ground_truth <ndarray>  : True Class labels  
        predictions <ndarray>   : Predicted probabilities
        k <int>                 : Rank
    
        returns
        -------
        ndcg_score <float> : NDCG@5 score
        """    
    
        # Compute relevance values for ground_truth        
        l=joblib.load("label_binarizer.pkl")
        T=l.transform(ground_truth)
    
        # Compute NDCG@k score for all samples
        scores = []
        for y_true, y_score in zip(T, predictions):
        
            dcg_k = dcg_score( y_true, y_score, k)
            idcg_k = dcg_score( y_true, y_true, k)
            ndcg_k = float(dcg_k) / float(idcg_k)
            scores.append(ndcg_k)
        
        # Mean of all scores
        ndcg_score=np.mean(scores)
    
        return ndcg_score
    score=ndcg_score(y_true,y_pred)
        
    return score


In [55]:
def data(train_data,sessions_data,sample_size):
    """
    This function is used to generate sample data from train and sessions data to check for predictions
    
    Function parameters:
    -------------------
    train_data: Input train/test data
    sessions_data: sessions data of users
    sample_size: size of samples from input data
    
    Returns:
    --------
    X:list of Dataframe of train with or without sessions data
    Y:True labels
    """
    #
    if isinstance(sessions_data,pd.DataFrame)==True:
        
        #to identify unique users of train data and sessions data
        a=train_data['id'].unique()
        b=sessions_data['user_id'].unique()
        
        random.shuffle(a)
        random.shuffle(b)
        
        #to take same users of traindata and sessions data
        lst=[]
        for i in b:
            if i in a:
                if len(lst)==sample_size:
                    break
                lst.append(i)
        
        #as train data and sessions have different i/p size/rows/examples we will concatenate one by one and store it in t
        df1=train_data.loc[train_data['id']==lst[0]]
        df2=sessions_data.loc[sessions_data['user_id']==lst[0]]

        for i,j in enumerate(lst):
            if i!=0:
                a=train_data.loc[train_data['id']==j]
                b=sessions_data.loc[sessions_data['user_id']==j]
                df1=pd.concat((df1,a),axis=0)
                df2=pd.concat((df2,b),axis=0)
        t=df1.drop(['country_destination'],axis=1)
        X=[t,df2]
        Y=df1['country_destination']
    else:
        #without sessions data
        a=train_data.sample(sample_size)
        b=a['country_destination']
        a=a.drop(['country_destination'],axis=1)
        X=[a,[]]
        Y=b
    
    return X,Y

# FUNCTION-1:

## SINGLE POINT

### a. WITH SESSIONS DATA

In [64]:
#with sessions data(for single point)
X,Y=data(train_data,sessions_data,1)
predictions_with_session=function_1(X)

In [65]:
#top5 predictions 
enc=joblib.load('label_enc.pkl')
enc.inverse_transform(np.argsort(predictions_with_session[0])[::-1][:5]).tolist()

['NDF', 'US', 'other', 'FR', 'GB']

### b.WITHOUT SESSIONS DATA

In [66]:
#without sessions data(for single point)
X,Y=data(train_data,[],1)
predictions_without_session=function_1(X)

In [67]:
#top5 predictions 
enc=joblib.load('label_enc.pkl')
enc.inverse_transform(np.argsort(predictions_without_session[0])[::-1][:5]).tolist()

['NDF', 'US', 'other', 'FR', 'IT']

## RANDOM SET OF POINTS

### a. WITH SESSIONS DATA

In [68]:
#with sessions data(for random set of points)
X,Y=data(train_data,sessions_data,30)
predictions_with_session_pts=function_1(X)

In [82]:
#top5 predictions 
enc=joblib.load('label_enc.pkl')
for i in range(predictions_with_session_pts.shape[0]):
    x=enc.inverse_transform(np.argsort(predictions_with_session_pts[i])[::-1][:5]).tolist()
    print("sample point",i,"th","top5 predictions",x)

sample point 0 th top5 predictions ['NDF', 'US', 'other', 'FR', 'GB']
sample point 1 th top5 predictions ['NDF', 'US', 'other', 'FR', 'IT']
sample point 2 th top5 predictions ['NDF', 'US', 'other', 'FR', 'IT']
sample point 3 th top5 predictions ['US', 'NDF', 'GB', 'other', 'FR']
sample point 4 th top5 predictions ['NDF', 'US', 'other', 'FR', 'IT']
sample point 5 th top5 predictions ['US', 'NDF', 'other', 'FR', 'IT']
sample point 6 th top5 predictions ['NDF', 'US', 'IT', 'FR', 'other']
sample point 7 th top5 predictions ['NDF', 'US', 'other', 'FR', 'GB']
sample point 8 th top5 predictions ['NDF', 'US', 'other', 'IT', 'FR']
sample point 9 th top5 predictions ['US', 'NDF', 'other', 'FR', 'ES']
sample point 10 th top5 predictions ['US', 'NDF', 'other', 'NL', 'FR']
sample point 11 th top5 predictions ['US', 'NDF', 'other', 'FR', 'IT']
sample point 12 th top5 predictions ['US', 'NDF', 'other', 'FR', 'IT']
sample point 13 th top5 predictions ['NDF', 'US', 'other', 'FR', 'IT']
sample point 14 

### b.WITHOUT SESSIONS DATA

In [88]:
#without sessions data(for random set of points)
X,Y=data(train_data,[],30)
predictions_without_session_pts=function_1(X)

In [89]:
#top5 predictions 
enc=joblib.load('label_enc.pkl')
for i in range(predictions_without_session_pts.shape[0]):
    x=enc.inverse_transform(np.argsort(predictions_without_session_pts[i])[::-1][:5]).tolist()
    print("sample point",i,"th","top5 predictions",x)

sample point 0 th top5 predictions ['NDF', 'US', 'other', 'FR', 'IT']
sample point 1 th top5 predictions ['NDF', 'US', 'other', 'FR', 'IT']
sample point 2 th top5 predictions ['NDF', 'US', 'other', 'FR', 'IT']
sample point 3 th top5 predictions ['NDF', 'US', 'other', 'FR', 'IT']
sample point 4 th top5 predictions ['NDF', 'US', 'other', 'FR', 'IT']
sample point 5 th top5 predictions ['NDF', 'US', 'other', 'FR', 'IT']
sample point 6 th top5 predictions ['NDF', 'US', 'other', 'FR', 'IT']
sample point 7 th top5 predictions ['NDF', 'US', 'other', 'FR', 'IT']
sample point 8 th top5 predictions ['NDF', 'US', 'other', 'FR', 'IT']
sample point 9 th top5 predictions ['NDF', 'US', 'other', 'FR', 'IT']
sample point 10 th top5 predictions ['NDF', 'US', 'other', 'FR', 'IT']
sample point 11 th top5 predictions ['NDF', 'US', 'other', 'FR', 'IT']
sample point 12 th top5 predictions ['NDF', 'US', 'other', 'FR', 'IT']
sample point 13 th top5 predictions ['NDF', 'US', 'other', 'FR', 'IT']
sample point 14 

# FUNCTION-2:

## FOR SINGLE POINT

### a.WITH SESSIONS DATA

In [56]:
#with sessions data(for single point)
X,Y=data(train_data,sessions_data,1)
function_2(X,Y)

1.0

### b.WITHOUT SESSIONS DATA

In [51]:
#without sessions data(for single point)
X,Y=data(train_data,[],1)
function_2(X,Y)

1.0

## FOR RANDOM SET OF POINTS

### a.WITH SESSIONS DATA

In [37]:
#with sessions data(for random set of points)
X,Y=data(train_data,sessions_data,30)
function_2(X,Y)

0.8548977212525178

### b.WITHOUT SESSIONS DATA

In [41]:
#without sessions data(for random set of points)
X,Y=data(train_data,[],30)
function_2(X,Y)

0.8805502758333401